In [1]:
import numpy as np
import sys
import os
import matplotlib.pyplot as plt
import copy
import torch as tr

# import scripts
from src.bayes_opt import bayesian_optimiser
from src.api_helper import api_utils
from src.api_helper import env

device = tr.device("cuda" if tr.cuda.is_available() else "cpu")
dtype = tr.float32

##### data

In [17]:
@api_utils.transform
def api(query): return env.rosenbrock(query)  # computation on cpu

x0, y0 = tr.tensor([4, 4], dtype=dtype).view(1, -1), tr.tensor([[-1]], dtype=dtype).view(-1, 1)  
r0 = env.rosenbrock(x0)  # initial reward on which normalisation based

x0, y0= x0.to(device), y0.to(device)  # data in GPU

print("init query", x0); print(f"init reward {api(x0, r0, device)}, origin reward: {r0.item():.2f}")  # x,y stay on GPU while r0 in cpu

init query tensor([[4., 4.]])
init reward tensor([[-1.]]), origin reward: 14409.00


##### OPT

In [18]:
# hyperparameters
T = 30  # total number of iterations

# gp; includes "MA2.5", "SE", "RQ", "LR", "PO"
gp_name, gp_params = "MA2.5",{
                          "mode": "raw",      # "raw", "add", "pro" for GPs
                          "opt":"ADAM",  # opt for MLE; (quasi_newton, ADAM)
                          "epochs":128,       # epoch to run, if chosen ADAM
                          "lr":1e-1,          # learning rate for ADAM
                         }
# q-parallelism (if use analytical acq_func, q must be 1)
batch_size = 2

acq_params = { 
    "acq_name" : "qEI",          # acqu func; includes: "EI", "UCB", "qEI", "qUCB", "qKG"
    "N_start": 32,               # number of starts for multi-start SGA
    "raw_samples" :512,          # heuristic initialisation 
    "N_MC_sample" : 256,         # number of samples for Monte Carlo simulation
    "num_fantasies": 128,        # number of fantasies used by KG
    "beta":1.,                   # used by UCB/qUCB
               }

bayes_opt = bayesian_optimiser(gp_name, gp_params, device, acq_params)

In [19]:
%%time
xs, ys = bayes_opt.outer_loop(T, (-4,4),x0, y0, r0, api, batch_size)  # maximising reward

NLML: -3.57
Iter: 1, reward: -0.41
NLML: 5.95
Iter: 2, reward: -0.34
NLML: -0.44
Iter: 3, reward: -0.12
NLML: -0.53
Iter: 4, reward: -0.00
NLML: -0.60
Iter: 5, reward: -0.00
NLML: -0.73
Iter: 6, reward: -0.00
NLML: -0.69
Iter: 7, reward: -0.00
NLML: -0.76
Iter: 8, reward: -0.00
NLML: -0.83
Iter: 9, reward: -0.00
NLML: -0.84
Iter: 10, reward: -0.00
NLML: -0.90
Iter: 11, reward: -0.00
NLML: -0.89
Iter: 12, reward: -0.00
NLML: -1.03
Iter: 13, reward: -0.01
NLML: -1.06
Iter: 14, reward: -0.00
NLML: -1.20
Iter: 15, reward: -0.00
NLML: -1.30
Iter: 16, reward: -0.00
NLML: -1.43
Iter: 17, reward: -0.00
NLML: -1.54
Iter: 18, reward: -0.00
NLML: -1.62
Iter: 19, reward: -0.00
NLML: -1.71
Iter: 20, reward: -0.00
NLML: -1.80
Iter: 21, reward: -0.00
NLML: -1.88
Iter: 22, reward: -0.00
NLML: -1.95
Iter: 23, reward: -0.00
NLML: -1.99
Iter: 24, reward: -0.00
NLML: -2.05
Iter: 25, reward: -0.00
NLML: -2.09
Iter: 26, reward: -0.00
NLML: -2.14
Iter: 27, reward: -0.00
NLML: -2.10
Iter: 28, reward: -0.00
NL

In [20]:
# from src.helper import helper
# saver.standard_save(xs, ys, r0, "bo", "./data/bo.pt")

saving bo data
